In [21]:
data_test = pd.read_excel('Data_Test.xlsx')

In [22]:
import pandas as pd
brand_name=[]
model_name=[]
for index, rows in data_test.iterrows():
    #print(rows['Name'].split())
    brand_name.append(rows['Name'].split()[0].lower().replace('-',''))
    model_name.append(rows['Name'].split()[1].lower())

In [23]:
data_test['Brand']=brand_name

In [24]:
for i in data_test.columns:
    if len(data_test[data_test[i].isnull()].index)>0:
        print(i,' Has Null values count : ',len(data_test[data_test[i].isnull()].index))

Engine  Has Null values count :  10
Power  Has Null values count :  10
Seats  Has Null values count :  11
New_Price  Has Null values count :  1052


In [25]:
data_test['is_electric']=data_test['Fuel_Type'].apply(lambda x: 1 if x=='Electric' else 0)    

In [26]:
Mileage_cleaned=[]
Engine_cleaned=[]
power_cleaned=[]
for index, rows in data_test.iterrows():
    #print(rows['Name'].split())
    try:
        Mileage_cleaned.append(rows['Mileage'].split()[0])
    except:
        Mileage_cleaned.append(0)
    try:
        Engine_cleaned.append(rows['Engine'].split()[0])
    except:
        Engine_cleaned.append(0)
    try:
        if rows['Power'].split()[0]=='null':
            power_cleaned.append(0)
        else:
            power_cleaned.append(rows['Power'].split()[0])
    except:
        power_cleaned.append(0)
    #print(rows['Mileage'].split()[1].lower())

In [27]:
data_test['Mileage_cleaned']=Mileage_cleaned
data_test.Mileage_cleaned = data_test.Mileage_cleaned.astype(float)
data_test['Engine_cleaned']=Engine_cleaned
data_test.Engine_cleaned = data_test.Engine_cleaned.astype(float)
data_test['Power_cleaned']=power_cleaned
data_test.Power_cleaned = data_test.Power_cleaned.astype(float)

In [28]:
Mileage_missing=[]
for i,j in data_test[(data_test['Mileage_cleaned']==0)].iterrows():
    if j.Engine_cleaned!=0:
        if j.Power_cleaned!=0:
            Mileage_missing.append(data_test[(data_test.Power_cleaned.between(j.Power_cleaned-50,j.Power_cleaned+50)) & (data_test.Engine_cleaned.between(j.Engine_cleaned-100,j.Engine_cleaned+100))]['Mileage_cleaned'].mean())
        else:
            Mileage_missing.append(data_test[(data_test.Engine_cleaned.between(j.Engine_cleaned-100,j.Engine_cleaned+100))]['Mileage_cleaned'].mean())
    else:
        if j.Power_cleaned!=0:
            Mileage_missing.append(data_test[(data_test.Power_cleaned.between(j.Power_cleaned-50,j.Power_cleaned+50))]['Mileage_cleaned'].mean())
        else:
            Mileage_missing.append(0)

In [29]:
j=0
for i in data_test[(data_test['Mileage_cleaned']==0)]['Mileage_cleaned'].index.tolist():
    data_test['Mileage_cleaned'].iloc[i]=Mileage_missing[j]
    j+=1

In [30]:
data_test.Mileage_cleaned[data_test.Mileage_cleaned == 0] = data_test['Mileage_cleaned'].mean()

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
for i in data_test.columns:
    if len(data_test[data_test[i].isnull()].index)>0:
        print(i,' Has Null values count : ',len(data_test[data_test[i].isnull()].index))

Engine  Has Null values count :  10
Power  Has Null values count :  10
Seats  Has Null values count :  11
New_Price  Has Null values count :  1052


In [32]:
P_M = data_test[(data_test['Fuel_Type']=='Petrol') & (data_test['Transmission']=='Manual')].Engine_cleaned.mean()
D_M = data_test[(data_test['Fuel_Type']=='Diesel') & (data_test['Transmission']=='Manual')].Engine_cleaned.mean()
D_A = data_test[(data_test['Fuel_Type']=='Diesel') & (data_test['Transmission']=='Automatic')].Engine_cleaned.mean()
P_A = data_test[(data_test['Fuel_Type']=='Petrol') & (data_test['Transmission']=='Automatic')].Engine_cleaned.mean()

data_test.Engine_cleaned.loc[(data_test['Fuel_Type']=='Petrol') & (data_test['Transmission']=='Manual') & (data_test['Engine_cleaned']==0)] = P_M
data_test.Engine_cleaned.loc[(data_test['Fuel_Type']=='Diesel') & (data_test['Transmission']=='Manual') & (data_test['Engine_cleaned']==0)] = D_M
data_test.Engine_cleaned.loc[(data_test['Fuel_Type']=='Diesel') & (data_test['Transmission']=='Automatic') & (data_test['Engine_cleaned']==0)] = D_A
data_test.Engine_cleaned.loc[(data_test['Fuel_Type']=='Petrol') & (data_test['Transmission']=='Automatic') & (data_test['Engine_cleaned']==0)] = P_A

In [33]:
Power_missing=[]
for i,j in data_test[(data_test['Power_cleaned']==0)].iterrows():
    Power_missing.append(data_test[(data_test.Mileage_cleaned.between(j.Mileage_cleaned-20,j.Mileage_cleaned+20)) 
                                      & (data_test.Engine_cleaned.between(j.Engine_cleaned-200,j.Engine_cleaned+200))]
                           ['Power_cleaned'].mean())

In [34]:
j=0
for i in data_test[(data_test['Power_cleaned']==0)]['Power_cleaned'].index.tolist():
    data_test['Power_cleaned'].iloc[i]=Power_missing[j]
    j+=1

In [35]:
seats_missing=[]
for i,j in data_test[(data_test.Seats.isnull()) | (data_test.Seats== 0)].iterrows():
    #data_train[data_train['Name'].str.match('Maruti Swift')]
    seats_missing.append(round(data_test[data_test['Name'].str.match(" ".join((j['Name'].split()[0],j['Name'].split()[1])))]['Seats'].mean()))
    #print(" ".join((j['Name'].split()[0],j['Name'].split()[1])))

In [36]:
j=0
for i in data_test[(data_test.Seats.isnull()) | (data_test.Seats== 0)].index.tolist():
    data_test['Seats'].iloc[i]=seats_missing[j]
    j+=1

In [37]:
data_test.Seats[data_test.Seats == 0] = data_test['Seats'].mean()

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
data_test.to_pickle('data_test_after_cleanup')